![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Spark NLP
### Multi-class Text Classification
#### By using ClassifierDL

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/ClassifierDL_Train_multi_class_news_category_classifier.ipynb)

Only run this block if you are inside Google Colab otherwise skip it

In [1]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed -q spark-nlp==2.4.5

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 58kB/s 
     |████████████████████████████████| 204kB 49.7MB/s 
     |████████████████████████████████| 112kB 2.8MB/s 


`UniversalSentenceEncoder` requires more `buffer.max` so we create the SparkSession manually:

In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
Apache Spark version


'2.4.4'

Let's download news category dataset for training our text classifier

In [3]:
!wget -O news_category_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_train.csv

--2020-05-12 16:48:29--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_train.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.200.125
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.200.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24032125 (23M) [text/csv]
Saving to: ‘news_category_train.csv’

news_category_train 100%[===================>]  22.92M  67.6MB/s    in 0.3s    

2020-05-12 16:48:29 (67.6 MB/s) - ‘news_category_train.csv’ saved [24032125/24032125]



In [4]:
!wget -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv

--2020-05-12 16:48:31--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.37
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1504408 (1.4M) [text/csv]
Saving to: ‘news_category_test.csv’

news_category_test. 100%[===================>]   1.43M  --.-KB/s    in 0.09s   

2020-05-12 16:48:31 (15.6 MB/s) - ‘news_category_test.csv’ saved [1504408/1504408]



In [5]:
!head news_category_train.csv

category,description
Business," Short sellers, Wall Street's dwindling band of ultra cynics, are seeing green again."
Business," Private investment firm Carlyle Group, which has a reputation for making well timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market."
Business, Soaring crude prices plus worries about the economy and the outlook for earnings are expected to hang over the stock market next week during the depth of the summer doldrums.
Business," Authorities have halted oil export flows from the main pipeline in southern Iraq after intelligence showed a rebel militia could strike infrastructure, an oil official said on Saturday."
Business," Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections."
Business," Stocks ended slightly higher on Friday but stayed near lows for the year as oil prices surged past  #36;

The content is inside `description` column and the labels are inside `category` column

In [ ]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("news_category_train.csv")

In [7]:
trainDataset.show()

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business| Short sellers, W...|
|Business| Private investme...|
|Business| Soaring crude pr...|
|Business| Authorities have...|
|Business| Tearaway world o...|
|Business| Stocks ended sli...|
|Business| Assets of the na...|
|Business| Retail sales bou...|
|Business|" After earning a...|
|Business| Short sellers, W...|
|Business| Soaring crude pr...|
|Business| OPEC can do noth...|
|Business| Non OPEC oil exp...|
|Business| WASHINGTON/NEW Y...|
|Business| The dollar tumbl...|
|Business|If you think you ...|
|Business|The purchasing po...|
|Business|There is little c...|
|Business|The US trade defi...|
|Business|Oil giant Shell c...|
+--------+--------------------+
only showing top 20 rows



In [8]:
trainDataset.count()

120000

In [ ]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [10]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("category")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
pipelineModel = pipeline.fit(trainDataset)

In [12]:
!cd ~/annotator_logs && ls -l

total 4
-rw-r--r-- 1 root root 527 May 12 16:56 ClassifierDLApproach_3ace16c99d52.log


In [13]:
!cat ~/annotator_logs/ClassifierDLApproach_d4a8d8ae15c4.log

cat: /root/annotator_logs/ClassifierDLApproach_d4a8d8ae15c4.log: No such file or directory


In [ ]:
from pyspark.sql.types import StringType

dfTest = spark.createDataFrame([
    "Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.",
    "Scientists have discovered irregular lumps beneath the icy surface of Jupiter's largest moon, Ganymede. These irregular masses may be rock formations, supported by Ganymede's icy shell for billions of years..."
], StringType()).toDF("description")

In [ ]:
prediction = pipelineModel.transform(dfTest)

In [16]:
prediction.select("class.result").show()

prediction.select("class.metadata").show(truncate=False)

+----------+
|    result|
+----------+
|[Business]|
|[Sci/Tech]|
+----------+

+------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                          |
+------------------------------------------------------------------------------------------------------------------+
|[[Sports -> 3.3033635E-7, Business -> 0.99999654, World -> 2.8160703E-6, Sci/Tech -> 2.9414142E-7, sentence -> 0]]|
|[[Sports -> 2.7236586E-15, Business -> 1.1168288E-14, World -> 7.367753E-14, Sci/Tech -> 1.0, sentence -> 0]]     |
+------------------------------------------------------------------------------------------------------------------+



In [ ]:
testDataset = spark.read \
      .option("header", True) \
      .csv("news_category_test.csv")

In [ ]:
preds = pipelineModel.transform(testDataset)

In [19]:
preds.select('category','description',"class.result").show(50, truncate=50)

+--------+--------------------------------------------------+----------+
|category|                                       description|    result|
+--------+--------------------------------------------------+----------+
|Business|Unions representing workers at Turner   Newall ...|[Business]|
|Sci/Tech| TORONTO, Canada    A second team of rocketeers...|[Sci/Tech]|
|Sci/Tech| A company founded by a chemistry researcher at...|[Sci/Tech]|
|Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|[Sci/Tech]|
|Sci/Tech| Southern California's smog fighting agency wen...|[Sci/Tech]|
|Sci/Tech|"The British Department for Education and Skill...|   [World]|
|Sci/Tech|"confessed author of the Netsky and Sasser viru...|[Sci/Tech]|
|Sci/Tech|\\FOAF/LOAF  and bloom filters have a lot of in...|[Sci/Tech]|
|Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|[Sci/Tech]|
|Sci/Tech|In its first two years, the UK's dedicated card...|[Sci/Tech]|
|Sci/Tech| A group of technology companies  includi

In [ ]:
preds_df = preds.select('category','description',"class.result").toPandas()

In [ ]:
# The result is an array since in Spark NLP you can have multiple sentences.
# This means you can add SentenceDetector in the pipeline and feed it into
# UniversalSentenceEncoder and you can have prediction based on each sentence.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

In [23]:
print (classification_report(preds_df['result'], preds_df['category']))

              precision    recall  f1-score   support

    Business       0.86      0.84      0.85      1942
    Sci/Tech       0.87      0.86      0.87      1928
      Sports       0.98      0.95      0.97      1948
       World       0.87      0.93      0.90      1782

    accuracy                           0.89      7600
   macro avg       0.89      0.89      0.89      7600
weighted avg       0.89      0.89      0.89      7600

